from https://www.kaggle.com/code/ohmeow/iterate-like-a-grandmaster-blurrified/notebook

In [2]:
! pip install -q ohmeow-blurr==1.0.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.3 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.3 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.3 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.3 requires wrapt~=1.12.1, but you have wrapt 1.14.0 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-serving-api 2.8.0 requires tensorflow<3,>=2.8.0, but you have tensorflow 2.6.3 which i

In [3]:
import gc

from fastai.callback.all import *
from fastai.data.block import CategoryBlock, ColReader, DataBlock, IndexSplitter, RegressionBlock
from fastai.imports import *
from fastai.learner import *
from fastai.optimizer import Adam
from fastai.metrics import *
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoModelForSequenceClassification, logging

from blurr.text.data.core import TextBlock
from blurr.text.modeling.core import BaseModelWrapper, BaseModelCallback, blurr_splitter
from blurr.text.utils import get_hf_objects
from blurr.utils import PreCalculatedCrossEntropyLoss, print_versions, set_seed

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
is_kaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")

In [5]:
if is_kaggle:
    path = Path("../input/us-patent-phrase-to-phrase-matching")
else:
    path = Path("./data")

path.ls()

(#3) [Path('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv'),Path('../input/us-patent-phrase-to-phrase-matching/train.csv'),Path('../input/us-patent-phrase-to-phrase-matching/test.csv')]

In [6]:
train_df = pd.read_csv(path/"train.csv")

print(len(train_df))
train_df.head()

36473


,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [7]:
test_df = pd.read_csv(path/"test.csv")

print(len(test_df))
test_df.head()

36


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [8]:
train_df['section'] = train_df.context.str[0]
train_df.section.value_counts()

B    8019
H    6195
G    6013
C    5288
A    4094
F    4054
E    1531
D    1279
Name: section, dtype: int64

In [9]:
model_checkpoint = "microsoft/deberta-v3-small" #"distilroberta-base"
batch_size = 128
weight_decay = 0.01

In [10]:
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_checkpoint, model_cls=AutoModelForSequenceClassification, config_kwargs={"num_labels": 1})

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

In [11]:
anchors = train_df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)

In [12]:
val_prop = 0.25
val_sz = int(len(anchors) * val_prop)
val_anchors = anchors[:val_sz]

In [13]:
is_val = np.isin(train_df.anchor, val_anchors)

idxs = np.arange(len(train_df))
val_idxs = idxs[ is_val]
trn_idxs = idxs[~is_val]

len(val_idxs),len(trn_idxs)

(9116, 27357)

In [14]:
train_df.iloc[trn_idxs].score.mean(), train_df.iloc[val_idxs].score.mean()

(0.3623021530138539, 0.3613426941641071)

In [15]:
set_seed(42)
sep = f' {hf_tokenizer.sep_token} '

def build_inputs(example):
    return f'{example["context"]}{sep}{example["anchor"]}{sep}{example["target"]}'

blocks = (TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), RegressionBlock)
dblock = DataBlock(blocks=blocks, get_x=build_inputs, get_y=ColReader("score"), splitter=IndexSplitter(val_idxs))

In [16]:
dls = dblock.dataloaders(train_df, bs=batch_size)

In [17]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

,text,target
0,C07 glycitin 3 4 hydroxyphenyl 6 methoxy 7 2s3r4s5s6r 345 trihydroxy 6 hydroxymethyloxan 2 yloxychromen 4 one,0.75
1,C08 rizatriptan 2 5 1h 124 triazol 1 ylmethyl 1h indol 3 yl nn dimethylethanamine,0.75


In [18]:
b = dls.one_batch()
len(b), len(b[0]["input_ids"]), b[0]["input_ids"].shape, len(b[1])

(2, 128, torch.Size([128, 43]), 128)

In [19]:
b[0]

{'input_ids': tensor([[   1,  716, 4649,  ...,  453,  311,    2],
         [   1,  716, 4649,  ...,    0,    0,    0],
         [   1,  802, 3085,  ...,    0,    0,    0],
         ...,
         [   1,  716, 3085,  ...,    0,    0,    0],
         [   1,  736, 3304,  ...,    0,    0,    0],
         [   1,  802, 3085,  ...,    0,    0,    0]], device='cuda:0'),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 'labels': tensor([0.7500, 0.5000, 0.7500, 0.5000, 0.7500, 0.0000, 0.2500, 0.2500, 1.0000,
         0.5000, 0.5000

In [20]:
def get_dls(df, get_x_func, batch_size, seed=42):
    set_seed(seed)

    blocks = (TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model), RegressionBlock)
    dblock = DataBlock(blocks=blocks, get_x=get_x_func, get_y=ColReader("score"), splitter=IndexSplitter(val_idxs))
    
    return dblock.dataloaders(df, bs=batch_size)

In [21]:
def get_learner(dls, hf_model, weight_decay=0.01, use_fp16=True, seed=42):
    set_seed(42)

    model = BaseModelWrapper(hf_model)

    learn = Learner(
        dls,
        model,
        opt_func=partial(Adam, wd=weight_decay),
        loss_func=PreCalculatedCrossEntropyLoss(),
        metrics=[PearsonCorrCoef()],
        cbs=[BaseModelCallback],
        splitter=blurr_splitter
    )

    if use_fp16:
        learn = learn.to_fp16()
        
    return learn

In [22]:
def build_inputs_with_sep(example, sep=" - ", lower_case=False):
    return f'{example["context"]}{sep}{example["anchor"]}{sep}{example["target"]}'

Looking at the best solution from the notebook

In [23]:
train_df['sectok'] = f'[{train_df.section}]'
sectoks = list(train_df.sectok.unique())

In [24]:
def build_inputs_with_sep_and_sectoks(example, sep=" - ", lower_case=False):
    return f'{example["sectok"]}{sep}{example["context"]}{sep}{example["anchor"]}{sep}{example["target"]}'

In [25]:
model_checkpoint = "microsoft/deberta-v3-small"
batch_size = 128
weight_decay = 0.01
use_fp16 = True
seed = 42

hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(model_checkpoint, model_cls=AutoModelForSequenceClassification, config_kwargs={"num_labels": 1})

# After adding the new tokens, we need to resize the embedding matrix in the model and initialize the weights
hf_tokenizer.add_special_tokens({'additional_special_tokens': sectoks})
hf_model.resize_token_embeddings(len(hf_tokenizer))

with torch.no_grad():
    hf_model.get_input_embeddings().weight[-len(hf_tokenizer), :] = torch.zeros([hf_model.config.hidden_size])

dls = get_dls(train_df, get_x_func=partial(build_inputs_with_sep_and_sectoks, sep=" [s] ", lower_case=True), batch_size=batch_size, seed=seed)
learn = get_learner(dls, hf_model, weight_decay=weight_decay, use_fp16=use_fp16, seed=seed)

learn.fit_one_cycle(4, lr_max=slice(1e-5, 1e-2))

epoch,train_loss,valid_loss,pearsonr,time
0,0.040246,0.025672,0.787620,01:07
1,0.028343,0.024718,0.805706,01:06
2,0.018687,0.025089,0.815104,01:06
3,0.014219,0.024142,0.817415,01:06
